# Vectorisation

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lukeconibear/swd6_hpp/blob/main/docs/03_vectorisation.ipynb)

In [3]:
# if you're using colab, then install the required modules
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    %pip install algorithms

## [Broadcasting](https://jakevdp.github.io/PythonDataScienceHandbook/02.05-computation-on-arrays-broadcasting.html)

Broadcasting allows for operations with different shaped arrays.

It's implemented in many libraries, such as [NumPy](https://numpy.org/doc/stable/user/basics.broadcasting.html) and [xarray](https://xarray.pydata.org/en/v0.16.2/computation.html?highlight=Broadcasting#broadcasting-by-dimension-name).

![broadcasting.png](images/broadcasting.png)  

*[Image source](https://mathematica.stackexchange.com/questions/99171/how-to-implement-the-general-array-broadcasting-method-from-numpy)*

In [174]:
import numpy as np

In [175]:
nums_col = np.array([0, 10, 20, 30]).reshape(4, 1)
nums_row = np.array([0, 1, 2])

nums_col + nums_row

array([[ 0,  1,  2],
       [10, 11, 12],
       [20, 21, 22],
       [30, 31, 32]])

In [176]:
import xarray as xr

In [177]:
nums_col = xr.DataArray([0, 10, 20, 30], [('col', [0, 10, 20, 30])])
nums_row = xr.DataArray([0, 1, 2], [('row', [0, 1, 2])])

nums_col + nums_row

<xarray.DataArray (col: 4, row: 3)>
array([[ 0,  1,  2],
       [10, 11, 12],
       [20, 21, 22],
       [30, 31, 32]])
Coordinates:
  * col      (col) int64 0 10 20 30
  * row      (row) int64 0 1 2

## [Vectorisation](https://jakevdp.github.io/PythonDataScienceHandbook/02.03-computation-on-arrays-ufuncs.html)

Vectorisation effectively "parallelises" the code by operating on multiple array elements at once, rather than looping through them one at a time.  

NumPy has many functions already vectorised for you, which have been optimised in C (i.e., they've been statically typed and compiled).

These are known as the universal functions ([ufuncs](https://numpy.org/doc/stable/reference/ufuncs.html)).

For example, instead of using `+`, you can use the equivalent ufunc `np.add`:

In [178]:
nums = np.arange(1_000_000)

In [179]:
%%timeit
[num + 2 for num in nums]

131 ms ± 15.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [182]:
%%timeit
nums + 2 # adds 2 to every element by overloading the + (similar to broadcasting)

691 µs ± 75.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [181]:
%%timeit
np.add(nums, 2)

682 µs ± 15.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Create your own ufunc

You can vectorise any arbitrary Python function to a NumPy ufunc using [`np.frompyfunc`](https://numpy.org/doc/stable/reference/generated/numpy.frompyfunc.html):

In [263]:
import math
SQRT_2PI = np.float32((2.0 * math.pi)**0.5)

def my_function(x, mean, sigma):
    '''Compute the value of a Gaussian probability density function at x with given mean and sigma.'''
    return math.exp(-0.5 * ((x - mean) / sigma)**2.0) / (sigma * SQRT_2PI)

In [264]:
x = np.random.uniform(-3.0, 3.0, size=1_000_000)

In [265]:
vectorized_function = np.frompyfunc(
    my_function, 
    nin=3, # number of input arguments 
    nout=1) # number of returned objects

In [266]:
%%timeit
vectorized_function(x, 0.0, 1.0)

1.77 s ± 318 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


However, `np.frompyfunc` removes any documentation:

In [267]:
vectorized_function.__doc__

"my_function (vectorized)(x1, x2, x3, /, out=None, *, where=True, casting='same_kind', order='K', dtype=None, subok=True[, signature, extobj])\n\ndynamic ufunc based on a python function"

To keep the documentation, instead use [`np.vectorize`](https://numpy.org/doc/stable/reference/generated/numpy.vectorize.html):

In [268]:
vectorized_function_with_docs = np.vectorize(my_function)

In [270]:
%%timeit
vectorized_function_with_docs(x, 0.0, 1.0)

1.75 s ± 246 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [271]:
vectorized_function_with_docs.__doc__

'Compute the value of a Gaussian probability density function at x with given mean and sigma.'

```{admonition} Question
Can you run `my_function` directly on the same inputs (i.e., x, 0.0, 1.0) without vectorising it?
```

```{admonition} Solution
:class: dropdown

No, a `TypeError` is returned.

This is because the function has not yet been vectorised, so it cannot operate across arrays with more than 1 element.

To check that it does work for 1 element, you could try: `my_function(x[0], 0.0, 1.0)`.

```

### Generalised universal functions ([gufuncs](https://numpy.org/doc/stable/reference/c-api/generalized-ufuncs.html))

Ufuncs apply the function element-by-element.

Whereas the generalized version (gufuncs) supports "sub-array" by "sub-array" operations. 

Numba has a nice implementation of these, which we will explore in the next lesson.

___

need to sort below ...

### [Lazy loading](https://xarray.pydata.org/en/v0.16.2/dask.html) and [execution](https://tutorial.dask.org/01x_lazy.html)
- Lazily loads metadata only, rather than eagerly loading data into memory.
- Creates task graph of scheduled work awaiting execution (`.compute()`).

In [1]:
import xarray as xr

In [2]:
xr.tutorial.open_dataset('air_temperature')

<xarray.Dataset>
Dimensions:  (lat: 25, time: 2920, lon: 53)
Coordinates:
  * lat      (lat) float32 75.0 72.5 70.0 67.5 65.0 ... 25.0 22.5 20.0 17.5 15.0
  * lon      (lon) float32 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
  * time     (time) datetime64[ns] 2013-01-01 ... 2014-12-31T18:00:00
Data variables:
    air      (time, lat, lon) float32 ...
Attributes:
    Conventions:  COARDS
    title:        4x daily NMC reanalysis (1948)
    description:  Data is from NMC initialized reanalysis\n(4x/day).  These a...
    platform:     Model
    references:   http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanaly...

### [Loop-invariants](https://en.wikipedia.org/wiki/Loop_invariant)
Move them *outside* the loop.  

Loops are slow in Python ([CPython](https://www.python.org/), default interpreter), because loops type−check and dispatch functions per cycle. Try to avoid them where can (e.g., using NumPy ufuncs, aggregations, broadcasting, etc.). 

In [1]:
%%timeit
for num in range(1_000_000):
    constant = 500_000
    bigger_num = max(num, constant)

144 ms ± 9.18 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [2]:
%%timeit
constant = 500_000
for num in range(1_000_000):
    bigger_num = max(num, constant)

139 ms ± 19.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Exercise

...

## Further information

### Other options

- ...

### Resources

- ...